# Review of Strassmann, Reinert and Dipankar
## (manuscript submited to QJ)
### Notes from the Associate Editor
The reviewer raises questions about the order of convergence of your implementation of PPM. I seem to spend a long time looking at why schemes are not converging with the expected order so I thought I would look into this with my own implementation of PPM. This raises an important point. The work presented in the manuscript is not reproducible. Not enough details were given so that I could be sure that I was reproducing the same test case for the constant, uniform velocity test in section 4.1, figure 2. Please double check the entire manuscript and ensure that enough detail is given so that an expert reader can reproduce all of your results, like I have tried to do. I have made some guesses below about how you implemented the convergence with resolution test.

My implementation, below, differs from yours in these ways:
1. I am using periodic boundary conditions. I suspected that this may help with the convergence.
2. I am sampling the initial conditions at grid points rather than the higher-order initialisation that you implemented. I think that this shouldn't harm the convergence because the analytic solution is sampled in the same way and the initial conditions are smooth.
3. I also tried using a sine wave as initial conditions as it is infinitely continuous with periodic boundary conditions.
4. I tried using a fixed Courant number of all simulations as well as a fixed time step, which I think was your approach.

To summarise my results, I am getting convergence around 3, similar to you.

# PPM From Colella dn Woodward (1984)

Method to solve 1D advection equation
$$
\frac{\partial a}{\partial t} + u \frac{\partial a}{\partial \xi} = 0
$$
This note describes the method for a uniform grid without monotonicity constraints.

$\xi_{j+1/2}$ is the boundary between cells $j$ and $j+1$.

$a_j^n$ is the average value of the solution in cell $j$.

$a(\xi)$ is a piecewise polynomial satisfying
$$
a_j^n = \frac{1}{\Delta\xi}\int_{\xi_{j-1/2}}^{\xi_{j+1/2}} a(\xi) d\xi
$$
\begin{eqnarray}
a(\xi) &=&a_{L,j} + x(\Delta a_j + a_{6,j}(1-x)) \\
x &=& \frac{\xi - \xi_{j-1/2}}{\Delta \xi}\\
\Delta a_j &=& a_{R,j} - a_{L,j}\\
a_{6,j} &=& 6\left(a_j^n - \frac{1}{2}(a_{L,j} + a_{R,j})\right)
\end{eqnarray}
Where the solution is smooth:
$$
a_{L,j+1} = a_{R,j} = a_{j+1/2}
$$
For a uniform grid:
$$
a_{j+1/2} = \frac{7}{12}(a_j^n + a_{j+1}^n) - \frac{1}{12}(a_{j+2}^n + a_{j-1}^n)
$$
Define
\begin{align*}
f_{j+1/2,L}^a(y) &= a_{R,j} - \frac{x}{2}\left(\Delta a_j - \left(1-\frac{2}{3}x\right)a_{6,j}\right), & \text{for } x&= \frac{y}{\Delta\xi} \\
f_{j+1/2,R}^a(y) &= a_{L,j+1} + \frac{x}{2}\left(\Delta a_{j+1} + \left(1-\frac{2}{3}x\right)a_{6,j+1}\right), & \text{for } x&= \frac{y}{\Delta\xi} \\
\end{align*}
Then
\begin{align*}
a_j^{n+1} &= a_j^n - u\frac{\Delta t}{\Delta \xi} (\overline{a}_{j+1/2} - \overline{a}_{j-1/2})&&\\
\text{where } \overline{a}_{j+1/2} &= f_{j+1/2,L}^a(u\Delta t)& \text{for } u\ge 0
\end{align*}

## Simplification for Smooth Solutions on a Uniform Grid with $u\ge 0$
Define Courant number, $c = u\Delta t/\Delta\xi = x$
\begin{align*}
a_j^{n+1} &= a_j^n - u\frac{\Delta t}{\Delta \xi} (\overline{a}_{j+1/2} - \overline{a}_{j-1/2})\\
a_{j+1/2} &= \frac{7}{12}(a_j^n + a_{j+1}^n) - \frac{1}{12}(a_{j+2}^n + a_{j-1}^n)\\
\overline{a}_{j+1/2} &=  a_{j+1/2} - \frac{c}{2}\left(a_{j+1/2} - a_{j-1/2} - \left(1-\frac{2}{3}c\right)6\left(a_j^n - \frac{1}{2}(a_{j-1/2} + a_{j+1/2})\right)\right)\\
     &=  \left(1 - \frac{c}{2} \right) a_{j+1/2}
      +  \left(\frac{c}{2} \right) a_{j-1/2}
      - \frac{c}{2}\left(  3-2c \right)\left(a_{j-1/2} + a_{j+1/2}\right)
      + c\left(3-2c\right)a_j^n\\
     &=  (1-c)^2 a_{j+1/2}
      +  c(1-c) a_{j-1/2}
      + c\left(3-2c\right)a_j^n
\end{align*}

# Large Courant numbers from LLM95
### (Leonard, Lock and Macvean, 1995)

$$
\overline{\phi}_j^{n+1} = \overline{\phi}_j^n - c(\phi_{jr} - \phi_{j\ell})
$$
Courant number, $c = u\Delta t/h = N +\Delta c$. Define $i = j-N$. Then
$$
\overline{\phi}_j^{n+1} = \overline{\phi}_{i}^n - \Delta c(\phi_{ir} - \phi_{i\ell})
$$
\begin{align*}
c \phi_{jr} &= \sum_{k=i+1}^j \overline{\phi}_k + \Delta c \phi_{ir}\\
\implies \phi_{jr} &= \frac{1}{c}\sum_{k=i+1}^j \overline{\phi}_k + \frac{\Delta c}{c} \phi_{ir}
\end{align*}


In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.dpi'] = 300
matplotlib.rcParams['figure.figsize'] =(8, 6)

In [ ]:
# 1D linear advection on a uniform, periodic domain with uniform, constant velocity
# A periodic domain is implimented using numpy roll.
# Interface j+1/2 is indexed j so that cell j is between interfaces indexed j-1 and j
# Assumes positive velocity so cell j is upwind of interface j-1
#         |  cell |   
#   j-1   |   j   |   j+1
#        j-1      j

# Generic advection from fluxes
def advect(phi, c, flux, options=None):
    """Advect cell values phi using Courant number c using flux for one time step.
    If "flux" is callable, it is called with arguments phi, c and options=options"""
    F = flux
    if callable(flux):
        F = flux(phi, c, options=options)
    return phi - c*(F - np.roll(F,1))

def PPMflux(phi, c,options=None):
    """Returns the PPM fluxes for cell values phi for Courant number c.
    Face j is at j+1/2 between cells j and j+1"""
    # Integer and remainder parts of the Courant number
    cI = int(c)
    cR = c - cI
    # phi interpolated onto faces
    phiI = 1/12*(-np.roll(phi,1) + 7*phi + 7*np.roll(phi,-1) - np.roll(phi,-2))
    # Move face interpolants to the departure faces
    if cI > 0:
        phiI = np.roll(phiI,cI)
    # Interface fluxes
    F = np.zeros_like(phi)
    # Contribution to the fluxes from full cells between the face and the departure point
    nx = len(F)
    for j in range(nx):
        for i in range(j-cI+1,j+1):
            F[j] += phi[i%nx]/c
    # Ratio of remnamt to total Courant number
    cS = 1
    if c > 1:
        cS = cR/c
    # Contribution from the departure cell
    F += cS*( (1 - 2*cR + cR**2)*phiI \
             + (3*cR - 2*cR**2)*np.roll(phi,cI) \
             + (-cR + cR**2)*np.roll(phiI,1))
    return F

In [ ]:
# Convergence test of Strassmann, Reinert and Dipankar (manuscript submited to QJ)
# My implimentation is different because I am using periodic boundary conditions
# and the initialisation is sampled at cell centres. This does not reduce the
# accuracy because the same method is used to find the analytic solution.

# Initial conditions (Gaussian and sine waves)
def initGauss(x):
    return np.exp(-(x - 0.5)**2/0.1**2)

def initSine(x):
    return np.sin(2*np.pi*x)

# Other parameters from Strassmann, Reinert and Dipankar
T = 0.01
cMax = 0.5
v = 1
# Range of numbers of grid points
nxs = 2**np.arange(5,11)
# The Time step is set from the knowledge that the maximum Courant number is given for the maximum n cells?
DT = cMax/nxs[-1]/v
# Number of time steps
NT = int(T/DT)+1
DT = T/NT
# The Courant number for each resolultion
cs = DT*v*nxs
print("Time step is", DT, "number of time steps is", NT, "Courant numbers are", cs)

# Different time steps for keeping the Courant number fixed at 0.5
T2 = 1/64
nts = 2*v*nxs*T2
nts = nts.astype('int32')
dts = T2/nts
print('For fixed Courant number', v*T2*nxs/nts, 'nts =', nts, 'T =', T2)

In [ ]:
# Solutions in comparison to analytic at the coarsest resolution
x = np.arange(0,1,1/nxs[0])
# Initial conditions and analytic solution
phi0 = initGauss(x)
phiE = initGauss((x-T)%1)
# Numerical solution
phiPPM = phi0.copy()
for it in range(NT):
    phiPPM = advect(phiPPM, cs[0], PPMflux)

# Plot solutions
plt.title('Advection using PPM with c = '+str(round(cs[0],2))+' nx = '
              +str(nxs[0]) + ' nt = '+str(NT))
plt.plot(x, phi0, 'k--', label = 't=0')
plt.plot(x, phiE, 'k', label='t='+str(round(DT*NT,2)))
plt.axhline(y=0, color='k', ls=':', lw=0.5)
plt.axhline(y=1, color='k', ls=':', lw=0.5)

plt.plot(x, phiPPM, 'b', label='PPM')
    
plt.legend()
plt.xlim([0,1])
plt.show()


In [ ]:
# Convergence with resolution using a fixed time step
errorsF = np.zeros_like(cs)
for i, nx, c in zip(range(len(nxs)), nxs, cs):
    x = np.arange(0,1,1/nx)
    # Initial conditions and analytic solution
    phi = initGauss(x)
    phiE = initGauss((x-T)%1)
    # Numerical solution
    for it in range(NT):
        phi = advect(phi, c, PPMflux)
    errorsF[i] = np.sqrt(np.sum((phi - phiE)**2)/np.sum(phiE**2))

# Convergence with resolution using a fixed Courant number
errorsC = np.zeros_like(dts)
for i, nx, nt in zip(range(len(nxs)), nxs, nts):
    c = T2*v*nx/nt
    x = np.arange(0,1,1/nx)
    # Initial conditions and analytic solution
    phi0 = initGauss(x)
    phiE = initGauss((x-T2)%1)
    # Numerical solution
    phi = phi0.copy()
    for it in range(nt):
        phi = advect(phi, c, PPMflux)
    errorsC[i] = np.sqrt(np.sum((phi - phiE)**2)/np.sum(phiE**2))

# Convergence with resolution using a fixed time step using sine initial conditions
errorsS = np.zeros_like(cs)
for i, nx, c in zip(range(len(nxs)), nxs, cs):
    x = np.arange(0,1,1/nx)
    # Initial conditions and analytic solution
    phi = initSine(x)
    phiE = initSine((x-T)%1)
    # Numerical solution
    for it in range(NT):
        phi = advect(phi, c, PPMflux)
    errorsS[i] = np.sqrt(np.sum((phi - phiE)**2)/np.sum(phiE**2))



In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1)
ax.loglog(nxs, errorsF, 'b-o', label='Gaussian, dt = '+str(round(DT,4)))
ax.loglog(nxs, errorsS, 'g-+', label='sin, dt = '+str(round(DT,4)))
ax.loglog(nxs, errorsC, 'r-x', label='Gaussian, c = '+str(round(c,4)))
ax.loglog(nxs, (1/nxs)**3*errorsF[-1]*nxs[-1]**3, 'k:', label='p=3')
ax.loglog(nxs, (1/nxs)**3*errorsS[-1]*nxs[-1]**3, 'k:')
ax.loglog(nxs, (1/nxs)**4*errorsF[-1]*nxs[-1]**4, 'k--', label='p=4')
ax.loglog(nxs, (1/nxs)**4*errorsS[-1]*nxs[-1]**4, 'k--')
ax.set_xscale('log', base=2)
plt.xlabel('Number of cells')
plt.ylabel(r'$L^2$ error')
ax.legend()
plt.show()